<a href="https://colab.research.google.com/github/luisaespinoza/CSCI167-Notebook12/blob/main/Copy_of_12_2_Multihead_Self_Attention.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Notebook 12.2: Multihead Self-Attention**

This notebook builds a multihead self-attention mechanism as in figure 12.6

Work through the cells below, running each cell in turn. In various places you will see the words "TODO". Follow the instructions at these places and make predictions about what is going to happen or write code to complete the functions.

Contact me at udlbookmail@gmail.com if you find any mistakes or have any suggestions.



In [10]:
import numpy as np
import matplotlib.pyplot as plt

The multihead self-attention mechanism maps $N$ inputs $\mathbf{x}_{n}\in\mathbb{R}^{D}$ and returns $N$ outputs $\mathbf{x}'_{n}\in \mathbb{R}^{D}$.  



In [11]:
# Set seed so we get the same random numbers
np.random.seed(3)
# Number of inputs
N = 6
# Number of dimensions of each input
D = 8
# Create an empty list
X = np.random.normal(size=(D,N))
# Print X
print(X)

[[ 1.789  0.437  0.096 -1.863 -0.277 -0.355]
 [-0.083 -0.627 -0.044 -0.477 -1.314  0.885]
 [ 0.881  1.71   0.05  -0.405 -0.545 -1.546]
 [ 0.982 -1.101 -1.185 -0.206  1.486  0.237]
 [-1.024 -0.713  0.625 -0.161 -0.769 -0.23 ]
 [ 0.745  1.976 -1.244 -0.626 -0.804 -2.419]
 [-0.924 -1.024  1.124 -0.132 -1.623  0.647]
 [-0.356 -1.743 -0.597 -0.589 -0.874  0.03 ]]


We'll use two heads.  We'll need the weights and biases for the keys, queries, and values (equations 12.2 and 12.4).  We'll use two heads, and (as in the figure), we'll make the queries keys and values of size D/H

In [12]:
# Number of heads
H = 2
# QDV dimension
H_D = int(D/H)

# Set seed so we get the same random numbers
np.random.seed(0)

# Choose random values for the parameters for the first head
omega_q1 = np.random.normal(size=(H_D,D))
omega_k1 = np.random.normal(size=(H_D,D))
omega_v1 = np.random.normal(size=(H_D,D))
beta_q1 = np.random.normal(size=(H_D,1))
beta_k1 = np.random.normal(size=(H_D,1))
beta_v1 = np.random.normal(size=(H_D,1))

# Choose random values for the parameters for the second head
omega_q2 = np.random.normal(size=(H_D,D))
omega_k2 = np.random.normal(size=(H_D,D))
omega_v2 = np.random.normal(size=(H_D,D))
beta_q2 = np.random.normal(size=(H_D,1))
beta_k2 = np.random.normal(size=(H_D,1))
beta_v2 = np.random.normal(size=(H_D,1))

# Choose random values for the parameters
omega_c = np.random.normal(size=(D,D))

Now let's compute the multiscale self-attention

In [13]:
# Define softmax operation that works independently on each column
def softmax_cols(data_in):
  # Exponentiate all of the values
  exp_values = np.exp(data_in) ;
  # Sum over columns
  denom = np.sum(exp_values, axis = 0);
  # Compute softmax (numpy broadcasts denominator to all rows automatically)
  softmax = exp_values / denom
  # return the answer
  return softmax

In [8]:
 # Now let's compute self attention in matrix form
def multihead_scaled_self_attention(X,omega_v1, omega_q1, omega_k1, beta_v1, beta_q1, beta_k1, omega_v2, omega_q2, omega_k2, beta_v2, beta_q2, beta_k2, omega_c):

  # Compute queries, keys, and values for head 1
  Q1 = omega_q1 @ X + beta_q1
  K1 = omega_k1 @ X + beta_k1
  V1 = omega_v1 @ X + beta_v1

  print("Q1:")
  print(Q1)
  print("K1:")
  print(K1)
  print("V1:")
  print(V1)

  # Compute attention scores for head 1
  attention_scores1 = Q1.T @ K1
  print("attention_scores1:")
  print(attention_scores1)


  # Scale attention scores for head 1
  scaled_attention_scores1 = attention_scores1 / np.sqrt(H_D)
  print("scaled_attention_scores1:")
  print(scaled_attention_scores1)


  # Apply softmax to get attention weights for head 1
  attention_weights1 = softmax_cols(scaled_attention_scores1)
  print("attention_weights1:")
  print(attention_weights1)


  # Compute the weighted sum of values for head 1
  head1_output = V1 @ attention_weights1
  print("head1_output:")
  print(head1_output)


  # Compute queries, keys, and values for head 2
  Q2 = omega_q2 @ X + beta_q2
  K2 = omega_k2 @ X + beta_k2
  V2 = omega_v2 @ X + beta_v2

  print("Q2:")
  print(Q2)
  print("K2:")
  print(K2)
  print("V2:")
  print(V2)

  # Compute attention scores for head 2
  attention_scores2 = Q2.T @ K2
  print("attention_scores2:")
  print(attention_scores2)


  # Scale attention scores for head 2
  scaled_attention_scores2 = attention_scores2 / np.sqrt(H_D)
  print("scaled_attention_scores2:")
  print(scaled_attention_scores2)


  # Apply softmax to get attention weights for head 2
  attention_weights2 = softmax_cols(scaled_attention_scores2)
  print("attention_weights2:")
  print(attention_weights2)


  # Compute the weighted sum of values for head 2
  head2_output = V2 @ attention_weights2
  print("head2_output:")
  print(head2_output)


  # Concatenate the outputs of the two heads
  concatenated_heads = np.vstack((head1_output, head2_output))
  print("concatenated_heads:")
  print(concatenated_heads)


  # Apply the final linear transformation
  X_prime = omega_c @ concatenated_heads
  print("X_prime before return:")
  print(X_prime)


  return X_prime

The comment in the code block below suggests looking at scaling if your values don't match. That said , it didn't neccessarily indicate a scaling method. I added print statements to track the value changes but after investigating at length, I've abandoned tracking down the issue.

In [15]:
# Run the self attention mechanism
X_prime = multihead_scaled_self_attention(X,omega_v1, omega_q1, omega_k1, beta_v1, beta_q1, beta_k1, omega_v2, omega_q2, omega_k2, beta_v2, beta_q2, beta_k2, omega_c)

# Print out the results
np.set_printoptions(precision=3)
print("Your answer:")
print(X_prime)

print("True values:")
print("[[-21.207  -5.373 -20.933  -9.179 -11.319 -17.812]")
print(" [ -1.995   7.906 -10.516   3.452   9.863  -7.24 ]")
print(" [  5.479   1.115   9.244   0.453   5.656   7.089]")
print(" [ -7.413  -7.416   0.363  -5.573  -6.736  -0.848]")
print(" [-11.261  -9.937  -4.848  -8.915 -13.378  -5.761]")
print(" [  3.548  10.036  -2.244   1.604  12.113  -2.557]")
print(" [  4.888  -5.814   2.407   3.228  -4.232   3.71 ]")
print(" [  1.248  18.894  -6.409   3.224  19.717  -5.629]]")

# If your answers don't match, then make sure that you are doing the scaling, and make sure the scaling value is correct

Q1:
[[ 2.733 -4.236  1.098 -4.049 -0.268  1.3  ]
 [ 1.905 -1.21   0.666  0.972  1.663  2.135]
 [ 4.82   5.904  0.313 -2.187 -0.775 -1.728]
 [ 3.686  3.582 -0.012 -4.523 -1.561 -5.601]]
K1:
[[ 0.408  4.393  0.7    3.87   4.135 -2.43 ]
 [-1.617 -6.555 -5.429  1.637  6.737  1.258]
 [-5.032 -6.485 -1.058  2.694  1.533  3.558]
 [ 1.935  3.734  1.504  2.489  3.995  2.556]]
V1:
[[-1.557 -4.362 -1.658 -0.806  0.242  1.597]
 [ 4.494  4.355  2.731  0.898  3.604  1.064]
 [-4.807  2.793  1.282 -0.021 -8.235 -4.459]
 [-1.971 -3.726 -3.472 -0.452 -1.94  -0.681]]
attention_scores1:
[[-19.088 -17.975  -7.989  35.855  46.25   22.328]
 [-22.549 -35.59    2.744   6.449  -2.305  38.935]
 [ -2.227  -1.616  -3.197   6.153   9.458  -0.749]
 [ -0.968 -26.86  -12.596 -31.228 -31.617  -8.282]
 [ -1.919 -12.882 -10.743  -4.292   2.665  -4.004]
 [ -5.063 -17.992 -17.274 -10.07   -5.267 -20.935]]
scaled_attention_scores1:
[[ -9.544  -8.988  -3.995  17.928  23.125  11.164]
 [-11.274 -17.795   1.372   3.224  -1.152 